<a href="https://colab.research.google.com/github/Sumetjutha/MyProject/blob/main/5_Build_A_Stock_Prediction_TMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install quandl

In [ ]:
# This program Predicts stock prices by nusing machine learning models
# Install the dependencies
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas_datareader import data
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = data.DataReader('TMB.bk',data_source="yahoo")
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-04,2.19477,2.13990,2.19477,2.13990,81077240.0,1.835067
2016-01-05,2.15819,2.10332,2.15819,2.12161,106283497.0,1.819382
2016-01-06,2.19477,2.12161,2.13990,2.17648,102702474.0,1.866436
2016-01-07,2.15819,2.12161,2.12161,2.12161,83287987.0,1.819382
2016-01-08,2.15819,2.12161,2.13990,2.15819,92758869.0,1.850752


In [ ]:
# Get the Adjusted Close Price
df = df[['Adj Close']]

In [ ]:
# Take a look at the new data
print(df.head())

            Adj Close
Date                 
2016-01-04   1.835067
2016-01-05   1.819382
2016-01-06   1.866436
2016-01-07   1.819382
2016-01-08   1.850752


In [ ]:
print(df.tail())

            Adj Close
Date                 
2020-12-24       1.13
2020-12-25       1.14
2020-12-28       1.10
2020-12-29       1.12
2020-12-30       1.08


In [ ]:
# A variable for prediction 'n' days out into the future
forecast_out = 1

In [ ]:
# Create anothr column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj Close']].shift(-forecast_out)

In [ ]:
print(df.head())

            Adj Close  Prediction
Date                             
2016-01-04   1.835067    1.819382
2016-01-05   1.819382    1.866436
2016-01-06   1.866436    1.819382
2016-01-07   1.819382    1.850752
2016-01-08   1.850752    1.788013


In [ ]:
print(df.tail())

            Adj Close  Prediction
Date                             
2020-12-24       1.13        1.14
2020-12-25       1.14        1.10
2020-12-28       1.10        1.12
2020-12-29       1.12        1.08
2020-12-30       1.08         NaN


In [ ]:
### Create the Independent data set (X)
# convert the data frame to a numpy array
X = np.array(df.drop(['Prediction'],1))

In [ ]:
# Remove the last 'n' rows 
X = X[:-forecast_out]

In [ ]:
print(X)

[[1.83506691]
 [1.81938207]
 [1.86643624]
 ...
 [1.13999999]
 [1.10000002]
 [1.12      ]]


In [ ]:
### Create the independent data set (y)
# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])

In [ ]:
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]

In [ ]:
print(y)

[1.81938207 1.86643624 1.81938207 ... 1.10000002 1.12       1.08000004]


In [ ]:
# Split the data into 80% training and 20% testting
x_train,x_test,y_train,y_test = train_test_split(X, y ,test_size=0.3)

In [ ]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)

In [ ]:
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test,y_test)

In [ ]:
print("svm confidence: ", svm_confidence)

svm confidence:  0.992047135073574


In [ ]:
# Create and trainLinear Regression Model
lr = LinearRegression()

In [ ]:
# Train the model
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
lr_confidence = lr.score(x_test,y_test)

In [ ]:
print("lr confidence: ", lr_confidence)

lr confidence:  0.9933464589601162


In [ ]:
# Set x_forecast equal to the last 30 rows of the original ata set from Adj Close columns
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]

In [ ]:
print(x_forecast)

[[1.08000004]]


In [ ]:
# Print the predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)

In [ ]:
print(lr_prediction)

[1.08037663]


In [ ]:
# Print the predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)

In [ ]:
print(svm_prediction)

[1.07849937]
